### Install packages and import

In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


### Define some functions

In [ ]:
# Function to check for pop-ups
def popup():
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Messaging popup
    try:
        messaging_soup = soup.find("li-icon", attrs={"type": "chevron-down-icon"})
        messaging_xpath = xpath_soup(messaging_soup)
        message_button = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_xpath(messaging_soup)
        )
        if message_button.is_enabled():
            print("Message popup detected.")
            message_button.click()
            print("Message popup dismissed.")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        "body > header > nav > section > section:nth-child(4) > form > button",
                    )
                )
            )
    except errors:

        # Accept cookies
        try:
            cookie_button = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_css_selector(
                    "#artdeco-global-alert-container > div.artdeco-global-alert.artdeco-global-alert--NOTICE.artdeco-global-alert--COOKIE_CONSENT > section > div > div.artdeco-global-alert-action__wrapper > button:nth-child(2)"
                )
            )
            if cookie_button.is_enabled():
                cookie_button.click()
                print("Cookies accepted.")
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (
                            By.CSS_SELECTOR,
                            "body > header > nav > section > section:nth-child(4) > form > button",
                        )
                    )
                )
        except errors:

            # Accept Alert
            try:
                click_alert = driver.switch_to.alert()
                print("Alert found.")
                click_alert.accept()
                print("Alert accepted.")
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (
                            By.CSS_SELECTOR,
                            "body > header > nav > section > section:nth-child(4) > form > button",
                        )
                    )
                )
            except errors:
                pass
            except:
                try:
                    print(f"Alert caused unexpected error: {sys.exc_info()[0]}")
                except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                    pass
        except:
            try:
                print(f"Cookies alert caused unexpected error: {sys.exc_info()[0]}")
            except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                pass
    except:
        try:
            print(f"Message popup caused unexpected error: {sys.exc_info()[0]}")
        except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
            pass

    return


In [ ]:
# Function to check for popups all the time
def popup_checker():
    while True:
        try:
            popup()
        except AttributeError:
            time.sleep(10)
            print(f"Unexpected error with popup checker: {sys.exc_info()[0]}.")


thread_popup_checker = multiprocessing.Process(target=popup_checker)


In [ ]:
# Function to find page number
def find_page_number(driver, soup, counter, keyword):
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        page_footer_soup = soup.find(
            "ul", class_="artdeco-pagination__pages artdeco-pagination__pages--number"
        )
        page_footer_xpath = xpath_soup(page_footer_soup)
        page_footer = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, page_footer_xpath))
        )
        if page_footer:
            page_number_length = (
                soup.find_all(
                    "li",
                    class_="artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view",
                )[-1]
                .find("button")
                .text.strip()
            )
            if int(page_number_length) == 1:
                print(f"Only 1 page found for {str(keyword)} jobs.")
                page_number = page_number_length
                page_number = [int(s) for s in page_number.split() if s.isdigit()][0]
                print(f"Current page number: {page_number}.")
            elif int(page_number_length) != 1:
                print(
                    f"Total of {page_number_length} pages found for {str(keyword)} jobs."
                )
                try:
                    page_number = (
                        soup.find(
                            "li",
                            class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                        )
                        .find("button", {"aria-current": "true"})
                        .text.strip()
                    )
                except:
                    page_number = (
                        soup.find(
                            "li",
                            class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                        )
                        .find(
                            "button", attrs={"aria-label": re.compile("*current page")}
                        )
                        .text.strip()
                    )
                page_number = [int(s) for s in page_number.split() if s.isdigit()][0]
                print(f"Current page number: {page_number}.")

        elif not page_footer:
            page_number = counter
            print(
                f"No page number indicator found. Page number set to counter: {page_number}"
            )
            pass
    except (NoSuchElementException, TimeoutException):
        page_number = counter
        page_number_length = 1
        print(
            f"No page number indicator found. Page number set to counter: {page_number}"
        )
        pass
    except errors:
        page_number = counter
        page_number_length = 1
        print(f"Unexpected error getting page number: {sys.exc_info()[0]}.")
        pass

    return page_number_length, page_number


In [ ]:
def get_page_jobs(
    driver,
    soup,
    counter,
    keyword,
    site,
    page_number,
    page_number_length,
    jobs,
    df_old_jobs,
    save_path,
    json_file_name,
):

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CLASS_NAME, "jobs-search-results"))
        )
        try:
            jobs_on_page = soup.find(
                "ul", class_="jobs-search-results__list list-style-none"
            ).find_all(
                "li",
                class_=re.compile("^jobs-search-results__list-item occludable-update*"),
            )
            jobs_count = len(jobs_on_page)
        except AttributeError as e:
            print(
                f"Unexpected error with job links: {e} - {sys.exc_info()[0]}.\nAttempting .children method."
            )
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (
                        By.CLASS_NAME,
                        "jobs-search-results__list-item occludable-update p0 relative ember-view",
                    )
                )
            )
            jobs_grid = soup.find_all(
                "ul", class_="jobs-search-results__list list-style-none"
            )
            jobs_on_page = [job.children for job in jobs_grid]
            jobs_count = cardinality.count(jobs_on_page)

        if int(jobs_count) != 0:
            print(
                f"{int(jobs_count)} job(s) found on page {page_number} out of {page_number_length} page(s)."
            )
            print(
                f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
            )

            for job_soup in jobs_on_page:
                try:
                    job_xpath = xpath_soup(job_soup)
                    job_button = WebDriverWait(driver, 10).until(
                        lambda driver: driver.find_element_by_xpath(job_xpath)
                    )

                    try:
                        job_button.click()
                    except errors:
                        time.sleep(5)
                        job_button.click()

                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (
                                By.CLASS_NAME,
                                "jobs-search-results__list-item occludable-update p0 relative ember-view",
                            )
                        )
                    )

                    # Get job id
                    job_id = job_soup.find(
                        "div",
                        class_=re.compile(
                            "^job-card-container relative job-card-list job-card-container--clickable*"
                        ),
                    )["data-job-id"]

                    # Check if already collected
                    jobs, counter, job_present = id_check(
                        driver,
                        counter,
                        keyword,
                        site,
                        df_old_jobs,
                        jobs,
                        job_soup,
                        job_id,
                        save_path,
                        json_file_name,
                    )

                    if job_present is False:
                        jobs, counter = get_each(
                            driver,
                            counter,
                            keyword,
                            site,
                            jobs,
                            job_soup,
                            job_id,
                            jobs_count,
                            page_number,
                            page_number_length,
                            save_path,
                            json_file_name,
                        )
                    elif job_present is True:
                        print(f"Increase counter\n  FROM: {counter}")
                        counter += 1
                        print(f"  TO: {counter}.")
                        pass

                    # If results open in new window, make sure everything loads
                    new_window = check_window(
                        driver, main_window, window_before, counter
                    )

                except:
                    print(
                        f"Problem getting job soup for {str(keyword)}: {sys.exc_info()[0]}. Moving on to next search."
                    )
                    # Get html of page with beautifulsoup4
                    html = driver.page_source
                    print("Feeding html driver to BeautifulSoup.")
                    soup = BeautifulSoup(html, "lxml")
                    jobs, jobs_count, counter = get_page_jobs(
                        driver,
                        soup,
                        counter,
                        keyword,
                        site,
                        page_number,
                        page_number_length,
                        jobs,
                        df_old_jobs,
                        save_path,
                        json_file_name,
                    )

        elif int(jobs_count) == 0:
            print(
                f"No jobs results found for {str(keyword)}. No error detected. Moving on to next search."
            )
    except:
        print(
            f"No jobs results found for {str(keyword)}: {sys.exc_info()[0]}. Moving on to next search."
        )
        pass
    finally:
        # Save dict as json file
        jobs = remove_dupe_dicts(jobs)
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
        print(f"Saving dict to json file as {json_file_name} in location {save_path}")
        with open(save_path + json_file_name, "w") as f:
            json.dump(jobs, f)

    return jobs, jobs_count, counter


In [ ]:
# Collect individual job links
def get_each(
    driver,
    counter,
    keyword,
    site,
    jobs,
    job_soup,
    job_id,
    jobs_count,
    page_number,
    page_number_length,
    save_path,
    json_file_name,
):
    jobs = remove_dupe_dicts(jobs)

    #     # If results open in new window, make sure everything loads
    #     new_window = check_window(driver, main_window, window_before, counter)

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Get job id, url and age
    print("-" * 20)
    print(f"Job ID: {job_id}")
    print("-" * 20)
    job_url = (
        "https://www.linkedin.com/"
        + job_soup.find(
            "div", class_="full-width artdeco-entity-lockup__title ember-view"
        ).find(
            "a",
            class_="disabled ember-view job-card-container__link job-card-list__title",
        )[
            "href"
        ]
    )
    print(f"Job URL: {job_url}")
    print("-" * 20)
    job_age = (
        job_soup.find(
            "li",
            class_=re.compile("^job-card-container__listed-time job-card-container*"),
        )
        .find("time")
        .text.strip()
    )
    for s in job_age.split():
        job_age_number = int(s) if s.isdigit() else job_age
    for i in re.findall("[\w +/.]", job_age):
        job_age_unit = str(i) if i.isalpha() else job_age
    print(f"Job Age: {job_age}")
    print("-" * 20)
    job_date = (
        job_soup.find(
            "li",
            class_=re.compile("^job-card-container__listed-time job-card-container*"),
        )
        .find("time")["datetime"]
        .strip()
    )
    print(f"Job Date: {job_date}")
    job_datetime = datetime.strptime(job_date, "%Y-%m-%d")
    print("-" * 20)

    #########################################################
    #########################################################
    ####################### TAB INFO #######################
    #########################################################
    #########################################################

    try:
        # Get main data
        (
            job_title,
            job_description,
            company_name,
            location,
            industry,
            sector,
            level,
            company_url,
            emp_type,
        ) = get_main(driver, soup)

    # In case there is no company tab, -1 for all variables
    except NoSuchElementException:
        print(f"No data collected: {sys.exc_info()[0]}.")
        job_title = -1
        job_description = -1
        company_name = -1
        location = -1
        industry = -1
        sector = -1
        level = -1
        company_url = -1
        emp_type = -1
        pass

    # Save to dict
    print("Saving to dict...")
    today = date.today()
    jobs.append(
        {
            "Search Keyword": str(keyword),
            "Platform": site,
            "Job ID": int(job_id),
            "Job Title": str(job_title),
            "Company Name": str(company_name),
            "Location": str(location),
            "Industry": str(industry),
            "Sector": str(sector),
            "Seniority Level": str(level),
            "Job Description": str(job_description),
            "Employment Type": str(emp_type),
            "Company URL": company_url,
            "Job URL": job_url,
            "Job Age": job_age,
            "Job Age Number": job_age_number,
            "Job Date": job_date,
            "Collection Date": (today.strftime("%Y-%m-%d")),
        }
    )
    jobs = remove_dupe_dicts(jobs)

    # Save dict as json file
    print(
        f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
    )
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Increase counter by 1
    print(f"Increase counter\n  FROM: {counter}")
    counter += 1
    print(f"  TO: {counter}.")
    with open(f"{os.getcwd()}/Data/Logs/counter_{keyword}.txt", "w") as f:
        f.write(str(counter))

    #     # Check current window and go back if required
    #     check_window_back(driver, main_window, window_before, new_window, counter)

    return jobs, counter


In [ ]:
# Function to get main job data
def get_main(driver, soup):

    pattern = re.compile(r"[^A-Za-z0-9, ]+")
    upper_pattern = re.compile("[A-Z][^A-Z\s]+(?:\s+\S[^A-Z\s]*)*")
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "jobs-search-results"))
    )
    job_container = soup.find("div", class_="jobs-details-top-card__container")
    job_container2 = soup.find("div", id="job-details").find(
        "div", class_="jobs-description-details pt4"
    )
    try:
        print("-" * 20)
        job_title = (
            job_container.find(
                "a", class_="jobs-details-top-card__job-title-link ember-view"
            )
            .find("h2", class_="jobs-details-top-card__job-title t-20 t-black t-normal")
            .text.strip()
        )
        print(f"Job Title: {job_title}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job title found.")
        job_title = None
    except errors:
        print(f"Unexpected error getting job title: {sys.exc_info()[0]}.")
        job_title = -1

    try:
        print("-" * 20)
        job_description = soup.find_all("div", id="job-details")
        #         for job_d in job_desc:
        # #             job_description = pattern.sub('', str(job_d.text.splitlines()))
        #             job_description = str(job_d.text.splitlines())
        #             job_description = job_description.strip().replace('  ', '').replace(', ,', '').replace(',,', '').replace('xa0', ' ').replace('&nbsp;', '\n').replace('Posted by', '')
        print(f"Job Description: {job_description[:50]}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job description found.")
        job_description = None
    except errors:
        print(f"Unexpected error getting job description: {sys.exc_info()[0]}.")
        job_description = -1

    try:
        print("-" * 20)
        #         company_name = pattern.sub('', job_container.find('div', class_ = 'jobs-details-top-card__content-container').find('a', class_ = re.compile('^jobs-details-top-card__company-url*')).text.strip())
        company_name = (
            job_container.find("div", class_="jobs-details-top-card__content-container")
            .find("a", class_=re.compile("^jobs-details-top-card__company-url*"))
            .text.strip()
        )
        print(f"Company Name: {company_name}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job company name found.")
        company_name = None
    except errors:
        print(f"Unexpected error getting company name: {sys.exc_info()[0]}.")
        company_name = -1

    try:
        print("-" * 20)
        #         location = pattern.sub('', soup.find('li', class_ = 'job-card-container__metadata-item').text.strip())
        location = soup.find(
            "li", class_="job-card-container__metadata-item"
        ).text.strip()
        print(f"Company Location: {location}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job company location found.")
        location = None
    except errors:
        print(f"Unexpected error getting company location: {sys.exc_info()[0]}.")
        location = -1

    try:
        print("-" * 20)
        industry_list = (
            job_container2.find(
                "h3", text=re.compile("Industry"), attrs={"class": "t-14 t-bold"}
            )
            .find_parent("div", class_="jobs-box__group")
            .find_all("ul", class_="jobs-box__list jobs-description-details__list")
        )
        for ind in industry_list:
            ind = ind.text.strip().replace("  ", "").split("\n")
            industry = [i for i in ind if i]
        print(f"Industry: {industry}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job company industry found.")
        industry = None
    except errors:
        print(f"Unexpected error getting industry: {sys.exc_info()[0]}.")
        industry = -1

    try:
        print("-" * 20)
        sector_list = (
            job_container2.find(
                "h3", text=re.compile("Job Functions"), attrs={"class": "t-14 t-bold"}
            )
            .find_parent("div", class_="jobs-box__group")
            .find_all("ul", class_="jobs-box__list jobs-description-details__list")
        )
        for sec in sector_list:
            sec = sec.text.strip().replace("  ", "").split("\n")
            sector = [s for s in sec if s]
        print(f"Sector: {sector}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job sector found.")
        sector = None
    except errors:
        print(f"Unexpected error getting sector: {sys.exc_info()[0]}.")
        sector = -1

    try:
        print("-" * 20)
        level = (
            job_container2.find(
                "h3", text=re.compile("Seniority Level"), attrs={"class": "t-14 t-bold"}
            )
            .find_parent("div", class_="jobs-box__group")
            .find("p", class_="t-14 mb3")
            .text.strip()
        )
        print(f"Seniority Level: {level}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job seniority level found.")
        level = None
    except errors:
        print(f"Unexpected error getting seniority level: {sys.exc_info()[0]}.")
        level = -1

    try:
        print("-" * 20)
        company_url = (
            "https://www.linkedin.com/"
            + job_container.find(
                "a", class_=re.compile("^jobs-details-top-card__company-url*")
            )["href"]
        )
        print(f"Company URL: {company_url}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job company url found.")
        company_url = None
    except errors:
        print(f"Unexpected error getting company url: {sys.exc_info()[0]}.")
        company_url = -1

    try:
        print("-" * 20)
        emp_type = (
            job_container2.find(
                "h3", text=re.compile("Employment Type"), attrs={"class": "t-14 t-bold"}
            )
            .find_parent("div", class_="jobs-box__group")
            .find("p", class_="t-14 mb3")
            .text.strip()
        )
        print(f"Employment Type: {emp_type}")
        print("-" * 20)
    except NoSuchElementException:
        print(f"No job employment type found.")
        emp_type = None
    except errors:
        print(f"Unexpected error getting employment type: {sys.exc_info()[0]}.")
        emp_type = -1

    if (
        all(
            [
                job_title,
                job_description,
                company_name,
                location,
                industry,
                sector,
                level,
                company_url,
                emp_type,
            ]
        )
        != -1
    ):
        print("Data collected successfully.")
        pass
    elif (
        any(
            [
                job_title,
                job_description,
                company_name,
                location,
                industry,
                sector,
                level,
                company_url,
                emp_type,
            ]
        )
        == -1
    ):
        print(
            f"Data NOT collected successfully. Sleeping for 10 seconds then trying again: {sys.exc_info()[0]}."
        )
        time.sleep(10)
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "#main-content > div > section > ul")
                )
            )
            (
                job_title,
                job_description,
                company_name,
                location,
                industry,
                sector,
                level,
                company_url,
                emp_type,
            ) = get_main(driver, soup)
        except:
            print(
                f"Data NOT collected successfully. Not attempting again: {sys.exc_info()[0]}."
            )
            pass

    return (
        job_title,
        job_description,
        company_name,
        location,
        industry,
        sector,
        level,
        company_url,
        emp_type,
    )


### Get-jobs function

In [ ]:
# MAIN FUNCTION TO GET JOB ADS
def get_jobs(
    keyword,
    keyword_url,
    site,
    driver,
    url,
    df_old_jobs,
    save_path,
    json_file_name,
    run_all,
):

    print(
        "-" * 20, f"BEGINING OF DATA COLLECTION FOR {keyword.upper()} JOB ADS", "-" * 20
    )
    main_window = driver.current_window_handle
    print(f"Main Window: {main_window}")
    driver.implicitly_wait(0)
    driver.set_window_size(1120, 1000)
    counter = 1
    see_more_counter = 0
    skipped_counters = []

    if is_non_zero_file(save_path + json_file_name) is True:
        print(
            f'A file with the name "{json_file_name}" already exists.\nNew data will be appended to this file.'
        )
        with open(save_path + json_file_name) as f:
            jobs = json.load(f)
            jobs = remove_dupe_dicts(jobs)
    elif is_non_zero_file(save_path + json_file_name) is False:
        jobs = []

    # Check internet connection
    if not is_connected():
        is_connected()

    # Go to page and establish main windows
    print(f"Going to url: {url}")
    driver.get(url)

    signed_out = WebDriverWait(driver, 10).until(
        lambda driver: driver.find_element_by_class_name("nav__cta-container")
    )
    if signed_out:
        # Sign in
        print("Signing in.")
        signin_button = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_class_name("nav__button-secondary")
        )
        try:
            signin_button.click()
        except (ElementNotInteractableException, ElementClickInterceptedException):
            time.sleep(1)
            signin_button.click()

        username = os.environ.get('YAHOO_EMAIL')
        password = os.environ.get('YAHOO_PASSWORD')
        username_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("username")
        )
        password_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("password")
        )
        username_textbox.send_keys(username)
        password_textbox.send_keys(password)
        password_textbox.send_keys(Keys.ENTER)
    else:
        print("Already signed in.")
        pass

    time.sleep(1)
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    app_content = soup.find("main", class_="app__content")
    if app_content != None:
        captcha = app_content.find("h1").text
        if captcha == "Let's do a quick security check":
            driver.close()
            driver.quit()
            initialize()
            time.sleep(5)
        #             run_all()
        elif captcha != "Let's do a quick security check":
            pass
    elif app_content is None:
        pass

    # Find page number
    page_number_length, page_number = find_page_number(driver, soup, counter, keyword)

    # Check for errors
    try:
        page_not_error = WebDriverWait(driver, 10).until_not(
            lambda driver: driver.find_element_by_class_name(
                re.compile("^jobs-search-no-results__image")
            )
        )

        while page_not_error:
            # Find job button next siblings with bs4
            print("Finding job links in page.")
            jobs, jobs_count, counter = get_page_jobs(
                driver,
                soup,
                counter,
                keyword,
                site,
                page_number,
                page_number_length,
                jobs,
                df_old_jobs,
                save_path,
                json_file_name,
            )

            # Click on next page button
            try:
                # Get html of page with beautifulsoup4
                html = driver.page_source
                print("Feeding html driver to BeautifulSoup.")
                soup = BeautifulSoup(html, "lxml")

                if int(page_number_length) == int(page_number):
                    print(
                        f"Last page reached at page {page_number}. End of data collection for {keyword}.\nGot {len(jobs)} jobs."
                    )
                    break
                elif int(page_number_length) != int(page_number):
                    next_page_soup = soup.find(
                        "li",
                        class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                    ).find_next_sibling(
                        "li",
                        attrs={
                            "class": re.compile(
                                "^artdeco-pagination__indicator artdeco-pagination__indicator--number*"
                            )
                        },
                    )
                    next_page_xpath = xpath_soup(next_page_soup)
                    next_page_button = driver.find_element_by_xpath(next_page_xpath)
                    next_page_button.click()
                    print(
                        f"Going to next page {int(page_number)+1}.\nGot {len(jobs)} jobs."
                    )
                    # Find page number
                    page_number_length, page_number = find_page_number(
                        driver, soup, counter
                    )
                    # Get jobs
                    jobs, jobs_count, counter = get_page_jobs(
                        driver,
                        soup,
                        counter,
                        keyword,
                        site,
                        page_number,
                        page_number_length,
                        jobs,
                        df_old_jobs,
                        save_path,
                        json_file_name,
                    )
            except (NoSuchWindowException, NoSuchElementException):
                jobs_count = len(jobs)
                print(
                    f"Could not go to next page. End of data collection for {keyword}.\nGot {len(jobs)} jobs."
                )
        else:
            jobs_count = len(jobs)
            print(f"No job results found for {str(keyword)}. Moving on to next search.")
    except TimeoutException:
        jobs_count = len(jobs)
        print(f"No jobs results found for {str(keyword)}. Moving on to next search.")

    ############################################### FINAL SAVING AND CLEANUP ###############################################

    # Save dict as json file
    print("Final save.")
    jobs = remove_dupe_dicts(jobs)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save jobs to df
    print(f"Saving {keyword} jobs data to df...")
    if jobs:
        df_jobs = pd.DataFrame(jobs)
        df_jobs.append(df_old_jobs)
        df_jobs = clean_df(df_jobs)
    elif not jobs:
        print(f"Jobs dict for {keyword} is empty.")
        df_jobs = pd.DataFrame()
        df_jobs.append(df_old_jobs)
        try:
            df_jobs = clean_df(df_jobs)
        except KeyError:
            pass

    # Save df as csv
    try:
        df_jobs, done = save_df(
            keyword,
            keywords_womenvocc,
            keywords_menvocc,
            keywords_sector,
            df_jobs,
            keyword_file.lower(),
            save_path.lower(),
            df_file_name.lower(),
        )
    except KeyError:
        pass

    return jobs, jobs_count, df_jobs, counter


### Run get_jobs function

In [ ]:
if __name__ == "__main__":

    site = "LinkedIn"
    language = "English"
    country = "Netherlands"  # in Dutch
    incognito_enabled = True
    headless_enabled = False
    proxy_enabled = False

    # Get keywords
    try:
        (
            keywords_list,
            keywords_womenvocc,
            keywords_menvocc,
            keywords_sector,
        ) = get_keyword_list()

    except AttributeError:
        time.sleep(1)
        (
            keywords_list,
            keywords_womenvocc,
            keywords_menvocc,
            keywords_sector,
        ) = get_keyword_list()

    # Start driver
    driver = get_driver(
        select_driver, incognito_enabled, headless_enabled, proxy_enabled
    )

    # Start threads
    # if not thread_popup_checker.is_alive():
    #     thread_popup_checker.start()
    # if not thread_stale_element_checker.is_alive():
    #     thread_stale_element_checker.start()
    # if not thread_act_cool_checker.is_alive():
    #     thread_act_cool_checker.start()

    # keywords_list = ['teacher']
    # Start for-loop
    for keyword in keywords_list:
        counter = 1
        page_counter = 0

        # Check internet connection
        if not is_connected():
            is_connected()

        # Get main info
        (
            keyword_url,
            keyword_file,
            save_path,
            json_file_name,
            df_file_name,
            logs_file_name,
            filemode,
        ) = main_info(keyword, site)

        # Set up log
        writer = MyWriter(logs_file_name.lower(), filemode)
        sys.stdout = writer
        logger = logging.getLogger(__name__)
        logging.basicConfig(
            filename=writer.LOGS_PATH + logs_file_name.lower(),
            filemode=filemode,
            level=logging.CRITICAL,
            format="%(asctime)s:%(levelname)s: %(message)s",
        )

        # Load and merge existing dict and df
        print(f"All files will be saved in folder: {save_path}")
        jobs, df_old_jobs = load_merge_dict_df(save_path, df_file_name, json_file_name)

        # Beging data collection
        print(
            "-" * 20,
            f"BEGINING OF DATA COLLECTION FOR {keyword.upper()} JOB ADS",
            "-" * 20,
        )
        main_window = driver.current_window_handle
        print(f"Main Window: {main_window}")

        # Get jobs
        jobs, jobs_count, df_jobs, counter = get_jobs(
            keyword.lower(),
            keyword_url,
            site,
            country,
            driver,
            counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
        )

        # Save df as csv
        df_jobs, done = save_df(
            keyword,
            keywords_womenvocc,
            keywords_menvocc,
            keywords_sector,
            df_jobs,
            keyword_file.lower(),
            save_path.lower(),
            df_file_name.lower(),
        )
        print(
            "-" * 20, f"END OF DATA COLLECTION FOR {keyword.upper()} JOB ADS", "-" * 20
        )

    # Close and quit driver
    print(f"Closing driver.")
    driver.close()
    driver.quit()
    # sys.stdout.close()
    # sys.exit()


In [ ]:
# keyword = 'retail trade'
# site = 'Indeed'
# from_list = False
# df_jobs = post_cleanup(keyword, site, from_list)
